Ce fichier pour ajouter les vidéos du jour au fichier videos.csv
- concatener et dédoublonner avec le fichier vidéos.csv
- passer les étapes de nettoyage
- sauvegarder dans videos.csv

In [10]:
import pandas as pd
from datetime import date
import re

In [12]:
base_videos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18038 entries, 0 to 18037
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   index                  18038 non-null  int64  
 1   video_id               18038 non-null  object 
 2   title                  18038 non-null  object 
 3   description            11659 non-null  object 
 4   channel_id             18038 non-null  object 
 5   category_id            18038 non-null  int64  
 6   language               18038 non-null  object 
 7   channel                18038 non-null  object 
 8   published_at           18038 non-null  object 
 9   duration               18022 non-null  object 
 10  views                  18038 non-null  int64  
 11  likes                  18038 non-null  int64  
 12  comments               18038 non-null  int64  
 13  Engagement total       18038 non-null  int64  
 14  Taux d'engagement (%)  18038 non-null  float64
 15  du

In [13]:
# ajouter les vidéos du jour 
DATE = date.today()
base_videos =  pd.read_csv("videos.csv")
new_videos =  pd.read_csv(f"new_videos/{DATE}.csv")
df = pd.concat([base_videos, new_videos],ignore_index=True)
df = df.sort_values(by=['video_id', 'views']) # si en doublon, prendre la ligne avec + de views 
df.drop_duplicates(subset=['video_id'], keep="last", inplace=True)
df.reset_index(drop=True, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18041 entries, 0 to 18040
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   index                  16315 non-null  float64
 1   video_id               18041 non-null  object 
 2   title                  18041 non-null  object 
 3   description            11661 non-null  object 
 4   channel_id             18041 non-null  object 
 5   category_id            18041 non-null  int64  
 6   language               18038 non-null  object 
 7   channel                18041 non-null  object 
 8   published_at           18041 non-null  object 
 9   duration               18025 non-null  object 
 10  views                  18041 non-null  int64  
 11  likes                  18041 non-null  int64  
 12  comments               18041 non-null  int64  
 13  Engagement total       16315 non-null  float64
 14  Taux d'engagement (%)  16315 non-null  float64
 15  du

In [5]:
# langues: supprimer les lignes avec valeurs
df.dropna(subset="language",axis=0,inplace=True)

# langues: regrouper fr_fr, be_fr sous 'fr', pareil pour 'en'
df.loc[df['language'].str.contains('^fr'),'language'] = "fr"
df.loc[df['language'].str.contains('^en'),'language'] = "en"

In [6]:
# calculer le taux d'engagement pour voir en un coup d'oeil les vidéos ayant reçu le plus d'intéractions
df["Engagement total"] = df["likes"] + df["comments"]
df["Taux d'engagement (%)"] = (df["Engagement total"] / df["views"].replace(0, 1)) * 100

In [7]:
# transofrmer les durées (ici min/sec) en secondes (plus simple pour l'analyse

# 1. Conversion en timedelta (errors='coerce' transforme les formats invalides en NaT)
df['duration_td'] = pd.to_timedelta(df['duration'], errors='coerce')

# 2. Conversion en secondes
# .astype('Int64') permet de garder les NaN si nécessaire sans faire planter le code
df['Durée (s)'] = df['duration_td'].dt.total_seconds().astype('Int64')


print(df[['duration', 'Durée (s)']])

       duration  Durée (s)
0       PT17M7S       1027
1         PT54S         54
2        PT1M1S         61
3         PT12S         12
4      PT10M12S        612
...         ...        ...
18038     PT26S         26
18039  PT37M10S       2230
18040     PT19S         19
18041   PT1M28S         88
18042     PT54S         54

[18038 rows x 2 columns]


In [ ]:
# RECUPERATION HASHTAGS
# extraction des hashtags des colonnes title et description
text_title_description = df['title'].fillna('') + " " + df['description'].fillna('')
df['hashtags'] = text_title_description.apply(lambda x: list(set(re.findall(r"#\w+", x))) or None)

# exploser la colonne hashtag pour compter les occurences
df_exploded = df.explode('hashtags').copy()
df_exploded = df_exploded.dropna(subset=['hashtags'])

df_hashtags = df_exploded.groupby(['category_id', 'hashtags']).size().reset_index(name='count')
df_hashtags = df_hashtags.sort_values(['category_id', 'count'], ascending=[True, False])

df_hashtags.to_csv("video_hashtags.csv", index=False)

In [8]:
df.to_csv("videos.csv", index=False)

In [29]:
# colonnes avec valeurs nulls
df.isna().sum()

video_id                    0
title                       0
description              4638
channel_id                  0
category_id                 0
language                    0
channel                     0
published_at                0
duration                   16
views                       0
likes                       0
comments                    0
Engagement total            0
Taux d'engagement (%)       0
duration_td                16
Durée (s)                  16
Unnamed: 0               5775
dtype: int64

In [31]:
df.shape

(13794, 17)

In [35]:
df

,video_id,title,description,channel_id,category_id,language,channel,published_at,duration,views,likes,comments,Engagement total,Taux d'engagement (%),duration_td,Durée (s),Unnamed: 0
0,eVrbPNIHfHY,LES JEUNES EN SALLE DE SPORT💪🤣#humour #shorts,NaN,UCpsvoU1qL4WLR6jvh1CWjhw,22,fr,Le déraciné,2025-06-16T16:01:04Z,PT11S,2640830,49359,450,49809,1.886112,0 days 00:00:11,11,NaN
1,kMPxRclSRrE,Je ne suis qu’un homme!!! #humour #lol #love #...,NaN,UCcIk1sZMBTu91ZxUHLr-BOg,23,fr,Kévin Lévy,2025-12-14T16:31:40Z,PT26S,919082,8487,144,8631,0.939089,0 days 00:00:26,26,NaN
2,E9CnEeOdESk,Sacha Ferra – C'est son style,Sacha Ferra – C'est son style\n\n𝙀𝙉𝙑𝙄𝙀 𝘿'𝙀𝙉𝘾𝙊𝙍...,UCPHJfLg1VeNBsfzzTszpmmQ,23,fr,Montreux Comedy,2025-10-08T16:00:32Z,PT10M29S,143065,4670,81,4751,3.320868,0 days 00:10:29,629,NaN
3,9qnIkeqYWtE,Por favor comparte con tus amigos #pov #herram...,NaN,UCUnw7mc6iHy8vZe8mvMKlUA,22,en,Mai Nguyên,2026-01-22T10:03:17Z,PT58S,1522,47,15,62,4.073587,0 days 00:00:58,58,NaN
4,hTKWX0cQtiE,Le dialogue le plus WTF jamais entendu ? #humo...,"C'est un papier qui tombe à l'eau. Et il crie,...",UCIQs0qjGOmjV2jTkazCvXyQ,22,fr,comedyUp,2025-12-20T22:08:50Z,PT17S,523750,2404,3,2407,0.459570,0 days 00:00:17,17,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13826,vlT1LGtvM5o,Routine for flexibility now in my bio ✅ #footb...,NaN,UCR5oDhM18tC-xWhZo_EapOw,17,en,Kivenro,2025-06-14T21:01:00Z,PT43S,8798665,128057,352,128409,1.459415,0 days 00:00:43,43,NaN
13827,V5IeC3ntcSg,Les transformations chimiques - cours de secon...,📈Les transformations chimiques pour la classe ...,UCmyeFnJJZE7mCtvbQ8IJitQ,27,fr,Les génies des sciences,2023-01-10T18:30:26Z,PT9M24S,139099,3538,126,3664,2.634095,0 days 00:09:24,564,NaN
13828,Qkaco2atfQo,Tone your lower body with this full leg workou...,NaN,UCo1bpycul1Xhpbo3gkhBBww,22,en,CallmeUche,2024-04-09T22:30:37Z,PT25S,2433500,89749,881,90630,3.724265,0 days 00:00:25,25,NaN
13829,ihUUzigRlDY,LES SECRETS de la TRANSFORMATION PHYSIQUE et M...,OBJECTIF RING. Épisode 2 : La Métamorphose.\n\...,UCwVnmWM7fmu6fTZd0AMzufg,26,fr,Training Thérapie,2024-12-01T17:58:17Z,PT38M44S,131126,4729,131,4860,3.706359,0 days 00:38:44,2324,NaN
